In [62]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
import ast
sys.path.append('../../..')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/data/medg/misc/semeval_2010/medical-data/DDICorpus"
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.ddi_preprocess.preprocess2 import get_dataset_dataframe
from relation_extraction.data.ddi_preprocess.preprocess2 import tokenize

In [92]:
#df = get_dataset_dataframe(res('Train/DrugBank/'))
#df = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'))
#df = get_dataset_dataframe(res('Train/MedLine/'))
#df = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'))

In [93]:
df_train_drugbank = get_dataset_dataframe(res('Train/DrugBank/'), relation_extraction=True)

  2%|▏         | 14/569 [00:00<00:04, 131.87it/s]

total_files_to_read: 569  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Train/DrugBank/


100%|██████████| 569/569 [00:05<00:00, 108.71it/s]


In [97]:
df_train_medline = get_dataset_dataframe(res('Train/MedLine/'), relation_extraction=True)

 11%|█         | 15/142 [00:00<00:00, 142.70it/s]

total_files_to_read: 142  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Train/MedLine/


100%|██████████| 142/142 [00:00<00:00, 150.35it/s]


In [95]:
df_test_drugbank = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'))

  8%|▊         | 12/158 [00:00<00:01, 90.25it/s]

total_files_to_read: 158  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Test/test_for_ddi_extraction_task/DrugBank/


100%|██████████| 158/158 [00:01<00:00, 118.14it/s]


In [96]:
df_test_medline = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'))

100%|██████████| 33/33 [00:00<00:00, 172.88it/s]

total_files_to_read: 33  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Test/test_for_ddi_extraction_task/MedLine/


## Check for empty values

In [102]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        entity_number = row.entity_number
        e1 = entity_number[0]
        e2 = entity_number[1]
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        sentence_text = df.iloc[index].sentence_text
        tokenized_sentence = df.iloc[index].tokenized_sentence
        entity_number = df.iloc[index].entity_number
        empty_rows_array.append([index, e1, e2, sentence_text, tokenized_sentence, entity_number])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'e1', 'e2', 'sentence_text', 'tokenized_sentence', 'entity_number'])
    return empty_rows_array, new_df

In [103]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [107]:
get_empty_vals(df_test_medline)

([], Empty DataFrame
 Columns: [index_original, e1, e2, sentence_text, tokenized_sentence, entity_number]
 Index: [])

#### Now can write these into csv format

In [109]:
df_train_drugbank.to_csv(res('pre-processed2/extraction/Train/DrugBank/train-drugbank_fulldrugblinding.csv'), encoding='utf-8', index=False)
# this is only relevant to the non NER task
df_test_drugbank.to_csv(res('pre-processed2/extraction/Test/DrugBank/test-drugbank_fulldrugblinding.csv'), encoding='utf-8', index=False)
df_train_medline.to_csv(res('pre-processed2/extraction/Train/MedLine/train-medline_fulldrugblinding.csv'), encoding='utf-8', index=False)
df_test_medline.to_csv(res('pre-processed2/extraction/Test/MedLine/test-medline_fulldrugblinding.csv'), encoding='utf-8', index=False)

#### Write the files into .txt format

In Train DrugBank, there is one sentence without a pair. Did ```find . -name "*.xml" | xargs grep "Drug/Laboratory Test Interactions:"``` for that